In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

In [2]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['start_station','end_station','Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles',
                     'rush_hour','metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    bikeshare_machine['start_station'] = bikeshare_machine['start_station'].astype('category')
    bikeshare_machine['end_station'] = bikeshare_machine['end_station'].astype('category')
    bikeshare_machine['rush_hour'] = bikeshare_machine['rush_hour'].astype('category')
    col_names = ['start_station', 'end_station','member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles','rush_hour',
                'metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['rush_hour','member_type','holiday','work_day'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['start_station','end_station','season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [3]:
file = '/Users/matthewcassi/Documents/Bike-Sharing-Dataset/Bikeshare_Time_Prediction/Casual_RushMetro/landmarks.csv'
bs = set_data(file)

In [4]:
bs.columns

Index(['time_diff', 'temperature', 'humidity', 'windspeed', 'miles',
       'metro_dist', 'landmark_dist_start', 'landmark_dist_end', 'temp', 'hum',
       ...
       'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', 'weather_cat_1', 'weather_cat_2',
       'weather_cat_3'],
      dtype='object', length=303)

In [5]:
bs=bs.rename(columns = {'member_type_Registered':'member_type'})
bs.head()

,time_diff,temperature,humidity,windspeed,miles,metro_dist,landmark_dist_start,landmark_dist_end,temp,hum,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
840495,12.617,0.377500,0.718750,0.082092,1.053809,0.458140,0.434793,0.060280,9.742500,71.8750,...,0,0,0,1,0,0,0,1,0,0
1000503,6.617,0.425000,0.741250,0.342667,0.634647,0.165573,0.821243,1.329181,11.975000,74.1250,...,0,0,0,0,0,1,0,0,1,0
813376,17.650,0.356667,0.552917,0.212062,2.098516,0.129048,0.086349,0.306432,8.763349,55.2917,...,0,0,0,0,0,0,1,1,0,0
856070,4.800,0.470000,0.812917,0.197763,0.461967,0.210402,1.300777,1.082599,14.090000,81.2917,...,0,0,0,0,1,0,0,0,1,0
1164845,7.650,0.545000,0.728750,0.162317,0.812539,0.252381,1.193484,2.004944,17.615000,72.8750,...,0,1,0,0,0,0,0,1,0,0


### Model 1

In [6]:
remove_cols = ['work_day_1','season_1', 'season_2', 'season_3', 'season_4',
               'hum','temp','wind','member_type']
X1 = np.matrix(bs.drop(remove_cols, 1))
y1 = bs['member_type']

In [7]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.25)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((914885, 294), (304962, 294), (914885,), (304962,))

In [8]:
c_space = np.linspace(0.0001, 50, 20)
param_grid = {'C': c_space}

log3 = LogisticRegression()
log_gs2 = GridSearchCV(log3, param_grid, cv=3)
log_gs2.fit(X1_train, y1_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-04,   2.63167e+00,   5.26325e+00,   7.89482e+00,
         1.05264e+01,   1.31580e+01,   1.57895e+01,   1.84211e+01,
         2.10527e+01,   2.36843e+01,   2.63158e+01,   2.89474e+01,
         3.15790e+01,   3.42106e+01,   3.68421e+01,   3.94737e+01,
         4.21053e+01,   4.47369e+01,   4.73684e+01,   5.00000e+01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [9]:
pred3 = log_gs2.predict(X1_test)
score3 = log_gs2.score(X1_test, y1_test)
train_score3 = log_gs2.score(X1_train, y1_train)

In [10]:
score3, train_score3

(0.87766672569041393, 0.87736491471605726)

In [11]:
confusion_matrix(y1_test, pred3)

array([[ 19047,  31633],
       [  5674, 248608]])

In [12]:
print(classification_report(y1_test, pred3))

             precision    recall  f1-score   support

          0       0.77      0.38      0.51     50680
          1       0.89      0.98      0.93    254282

avg / total       0.87      0.88      0.86    304962



In [13]:
roc_auc_score(y1_test, pred3)

0.67675746010182891